<a href="https://colab.research.google.com/github/verma-saloni/Thesis-Work/blob/main/Copy_of_Clean_Fasttext_10_records.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [50]:
!pip install unidecode
!pip install word2number
!pip install contractions

In [51]:
import pandas as pd
import numpy as np
import nltk
import gensim
from gensim.models import Word2Vec
import string
string.punctuation
nltk.download('stopwords')
import re
from bs4 import BeautifulSoup
import spacy
import unidecode
from word2number import w2n
import contractions
from nltk.tokenize.treebank import TreebankWordDetokenizer
# load spacy model, can be "en_core_web_sm" as well
from sklearn.model_selection import train_test_split
import xgboost as xgb



[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [52]:
from google.colab import drive
drive._mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [53]:
df_fake = pd.read_csv('/content/drive/MyDrive/Thesis/gossipcop_fake.csv')
df_real = pd.read_csv('/content/drive/MyDrive/Thesis/gossipcop_real.csv')
df_fake['labelML']=0
df_real['labelML']=1

df_fake.count()


id           5323
news_url     5067
title        5323
tweet_ids    5135
labelML      5323
dtype: int64

In [54]:
dforiginal = df_fake.head(5).append(df_real.head(5))

In [55]:
#df= df_fake.head(5).append(df_real.head(5))
df= df_fake.append(df_real)
df.reset_index(drop=True, inplace=True)

In [56]:
#print(df.columns)
##df.count()
df

,id,news_url,title,tweet_ids,labelML
0,gossipcop-2493749932,www.dailymail.co.uk/tvshowbiz/article-5874213/...,Did Miley Cyrus and Liam Hemsworth secretly ge...,284329075902926848\t284332744559968256\t284335...,0
1,gossipcop-4580247171,hollywoodlife.com/2018/05/05/paris-jackson-car...,Paris Jackson & Cara Delevingne Enjoy Night Ou...,992895508267130880\t992897935418503169\t992899...,0
2,gossipcop-941805037,variety.com/2017/biz/news/tax-march-donald-tru...,Celebrities Join Tax March in Protest of Donal...,853359353532829696\t853359576543920128\t853359...,0
3,gossipcop-2547891536,www.dailymail.co.uk/femail/article-3499192/Do-...,Cindy Crawford's daughter Kaia Gerber wears a ...,988821905196158981\t988824206556172288\t988825...,0
4,gossipcop-5476631226,variety.com/2018/film/news/list-2018-oscar-nom...,Full List of 2018 Oscar Nominations – Variety,955792793632432131\t955795063925301249\t955798...,0
...,...,...,...,...,...
22135,gossipcop-891749,https://omgcheckitout.com/2017-hollywood-film-...,2017 Hollywood Film Awards: The Complete List ...,927385466357260288\t927386601034010625\t927387...,1
22136,gossipcop-941486,https://pagesix.com/2018/06/04/jada-pinkett-sm...,Jada Pinkett Smith explains why son Jaden move...,1004044947006386178\t1004045964401889285\t1004...,1
22137,gossipcop-953143,https://www.etonline.com/tinsley-mortimer-reac...,Tinsley Mortimer Reacts to Luann de Lesseps' R...,1019924845889572864\t1019925702676709377\t1019...,1
22138,gossipcop-954366,https://www.healthbreakingnews.net/2018/07/pri...,Prince Harry Carries on Princess Diana’s Legac...,1021766291139584000\t1021772054599802880\t1021...,1


In [57]:
print(df)

                         id  ... labelML
0      gossipcop-2493749932  ...       0
1      gossipcop-4580247171  ...       0
2       gossipcop-941805037  ...       0
3      gossipcop-2547891536  ...       0
4      gossipcop-5476631226  ...       0
...                     ...  ...     ...
22135      gossipcop-891749  ...       1
22136      gossipcop-941486  ...       1
22137      gossipcop-953143  ...       1
22138      gossipcop-954366  ...       1
22139      gossipcop-877062  ...       1

[22140 rows x 5 columns]


In [58]:
df1= df.drop(['tweet_ids', 'news_url','id'], axis=1)
df1.count()

title      22140
labelML    22140
dtype: int64

In [59]:
# NLP Preprocessing
from gensim.utils import simple_preprocess

# NLP Preprocess - gensim.utils.simple_preprocess(doc, deacc=False, min_len=2, max_len=15)[source]
# Convert a document into a list of tokens.
# This lowercases, tokenizes, de-accents (optional). – the output are final tokens = unicode strings, that won’t be processed any further.

df1.iloc[:, 0] = df1.iloc[:, 0].apply(lambda x: ' '.join(simple_preprocess(x)))

In [60]:
print(df1)
#df1.count()

                                                   title  labelML
0      did miley cyrus and liam hemsworth secretly ge...        0
1      paris jackson cara delevingne enjoy night out ...        0
2      celebrities join tax march in protest of donal...        0
3      cindy crawford daughter kaia gerber wears wig ...        0
4                 full list of oscar nominations variety        0
...                                                  ...      ...
22135  hollywood film awards the complete list of win...        1
22136  jada pinkett smith explains why son jaden move...        1
22137  tinsley mortimer reacts to luann de lesseps re...        1
22138  prince harry carries on princess diana legacy ...        1
22139  kylie jenner is actually terrified of butterflies        1

[22140 rows x 2 columns]


In [61]:
# Prefixing each row of the category column with '__label__' 0;10 all rows. same for column. 
df1.iloc[:, 0] = df1.iloc[:, 0].apply(lambda x: '__label__ ' + x)
#df1.iloc[5:, 0] = df1.iloc[1:, 0].apply(lambda x: '__label__ ' + x)

# NOTE BIG NOTE In general it is important to properly preprocess your data. In particular our example scripts in the root folder do this. i can see why the last cell is wierd, fix in post.


In [62]:
print(df1)

                                                   title  labelML
0      __label__ did miley cyrus and liam hemsworth s...        0
1      __label__ paris jackson cara delevingne enjoy ...        0
2      __label__ celebrities join tax march in protes...        0
3      __label__ cindy crawford daughter kaia gerber ...        0
4       __label__ full list of oscar nominations variety        0
...                                                  ...      ...
22135  __label__ hollywood film awards the complete l...        1
22136  __label__ jada pinkett smith explains why son ...        1
22137  __label__ tinsley mortimer reacts to luann de ...        1
22138  __label__ prince harry carries on princess dia...        1
22139  __label__ kylie jenner is actually terrified o...        1

[22140 rows x 2 columns]


In [63]:
df1.count()

title      22140
labelML    22140
dtype: int64

In [64]:
df2=df1
# ------------- df2 = df2.reindex(columns=['labelML', 'title'])
#print(df1)

In [65]:
traindf, testdf = train_test_split(df2, test_size=0.2)
print(traindf)
print(testdf)  

traindf.count()

                                                   title  labelML
20961  __label__ bella thorne spills the tea on what ...        1
17481  __label__ kim kardashian goes for surprisingly...        1
11828  __label__ harvey weinstein scandal who has acc...        1
7132   __label__ heath ledger documentary what we lea...        1
15429  __label__ nastia liukin praises gymnasts for s...        1
...                                                  ...      ...
2668   __label__ ellen degeneres makes joke about jen...        0
17759            __label__ the best skincare products of        1
7128   __label__ mel says the spice girls were never ...        1
17423  __label__ lala kent thinks tupac took over her...        1
6677   __label__ photos of celebrities not wearing th...        1

[17712 rows x 2 columns]
                                                   title  labelML
19347  __label__ sons of anarchy actor alan neill die...        1
11535  __label__ serena williams shares an empower

title      17712
labelML    17712
dtype: int64

In [66]:
# Saving the CSV file as a text file to train/test the classifier
import csv #needed
traindf[['labelML', 'title']].to_csv('traindf.txt', 
                                          index = False, 
                                          sep = ' ',
                                          header = None, 
                                          quoting = csv.QUOTE_NONE, 
                                          quotechar = "", 
                                          escapechar = " ")

testdf[['labelML', 'title']].to_csv('testdf.txt', 
                                     index = False, 
                                     sep = ' ',
                                     header = None, 
                                     quoting = csv.QUOTE_NONE, 
                                     quotechar = "", 
                                     escapechar = " ")

In [67]:
!pip install fasttext
import fasttext
# Training the fastText classifier
model = fasttext.train_supervised('traindf.txt', wordNgrams = 2, dim=300)

# Evaluating performance on the entire test file
model.test('testdf.txt')                      

# Predicting on a single input
testdf['labelML'] = testdf['labelML'].astype(str)
model.predict(testdf.iloc[1, 0])
#model.test(testdf.txt)

# Save the trained model
#model.save_model('modelFTme.bin')

(('__label__',), array([1.00001001]))

In [68]:
testdf.head()
#print(testdf.columns)

,title,labelML
19347,__label__ sons of anarchy actor alan neill die...,1
11535,__label__ serena williams shares an empowering...,1
21328,__label__ cardi hides figure in voluminous fri...,1
5384,__label__ leonardo dicaprio makes first appear...,1
1008,__label__ jennifer aniston leaning on brad pit...,0


In [69]:
model.get_nearest_neighbors('hello')

[(0.22322112321853638, 'tara'),
 (0.21367326378822327, 'quittman'),
 (0.20864391326904297, 'xmas'),
 (0.20477765798568726, 'myspace'),
 (0.20362640917301178, 'amazing'),
 (0.20144712924957275, 'indecent'),
 (0.1942497193813324, 'delaney'),
 (0.1926089972257614, 'horribly'),
 (0.19060228765010834, 'lax'),
 (0.18584252893924713, 'amy')]

This model is running, so trying to change with vectors, or textual input for new vectors. Accuracy is only 50 percent coz I've given it such a balanced dataset.

In [70]:
embedding=[]
for line in df2['title']:
    pred_label=model.get_sentence_vector(line)
    embedding.append(pred_label)

# you add the list to the dataframe, then save the datframe to new csv
df2['prediction']=embedding
df2.to_csv('csv_file_w_pred.csv',index=False)

In [71]:
df2

,title,labelML,prediction
0,__label__ did miley cyrus and liam hemsworth s...,0,"[1.6669663e-05, 8.239681e-05, -0.00048171758, ..."
1,__label__ paris jackson cara delevingne enjoy ...,0,"[2.3270495e-05, 0.0004015812, -2.8936995e-06, ..."
2,__label__ celebrities join tax march in protes...,0,"[0.00058013183, 0.00024639844, -0.00029291608,..."
3,__label__ cindy crawford daughter kaia gerber ...,0,"[2.6209871e-05, -0.00026430548, 5.9815775e-06,..."
4,__label__ full list of oscar nominations variety,0,"[0.00012178716, 0.00052041665, -0.00043820584,..."
...,...,...,...
22135,__label__ hollywood film awards the complete l...,1,"[0.0003885683, 0.000119031356, -0.00029722182,..."
22136,__label__ jada pinkett smith explains why son ...,1,"[0.00031934853, 0.00033909114, -0.0006084566, ..."
22137,__label__ tinsley mortimer reacts to luann de ...,1,"[-5.974903e-05, 0.0003673786, -0.00039826173, ..."
22138,__label__ prince harry carries on princess dia...,1,"[-5.633611e-05, -0.00014275745, -0.00022375959..."


In [72]:
type(df2)

pandas.core.frame.DataFrame

In [73]:
df3 = df2.drop(['title'], axis=1)
df3

,labelML,prediction
0,0,"[1.6669663e-05, 8.239681e-05, -0.00048171758, ..."
1,0,"[2.3270495e-05, 0.0004015812, -2.8936995e-06, ..."
2,0,"[0.00058013183, 0.00024639844, -0.00029291608,..."
3,0,"[2.6209871e-05, -0.00026430548, 5.9815775e-06,..."
4,0,"[0.00012178716, 0.00052041665, -0.00043820584,..."
...,...,...
22135,1,"[0.0003885683, 0.000119031356, -0.00029722182,..."
22136,1,"[0.00031934853, 0.00033909114, -0.0006084566, ..."
22137,1,"[-5.974903e-05, 0.0003673786, -0.00039826173, ..."
22138,1,"[-5.633611e-05, -0.00014275745, -0.00022375959..."


In [74]:
for index, arr in df3['prediction'].iteritems():
  for i in range(len(arr)):
    df3.loc[index, str(i)] = arr[i]

In [75]:
df3.to_pickle('ourpickle.pkl', compression=None)

In [76]:
#df3['prediction','new'] = df3['prediction'].str.split(',', expand=True)

In [77]:
df3

,labelML,prediction,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,...,260,261,262,263,264,265,266,267,268,269,270,271,272,273,274,275,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,294,295,296,297,298,299
0,0,"[1.6669663e-05, 8.239681e-05, -0.00048171758, ...",0.000017,0.000082,-0.000482,-0.000154,-0.000103,-0.000379,0.000346,-0.000275,-0.000421,-0.000116,-0.000486,0.000187,0.000339,0.000233,-0.000007,-0.000046,-0.000188,-0.000018,0.000398,0.000003,-0.000449,-0.000175,0.000471,-0.000094,-0.000522,-0.000375,-0.000009,0.000200,-0.000313,0.000459,0.000497,-0.000295,-0.000117,0.000549,-0.000312,-0.000301,0.000705,-0.000385,...,0.000210,-0.000175,0.000439,0.000024,0.000045,-0.000325,-0.000295,-0.000032,-0.000013,0.000640,-0.000153,-0.000069,0.000425,-0.000482,-0.000416,0.000111,0.000459,0.000118,0.000067,0.000209,0.000547,0.000133,0.000342,0.000308,0.000140,0.000082,-0.000090,0.000570,-0.000101,-0.000414,0.000298,-0.000221,0.000105,-0.000021,-0.000309,-0.000605,0.000445,-0.000266,0.000030,0.000142
1,0,"[2.3270495e-05, 0.0004015812, -2.8936995e-06, ...",0.000023,0.000402,-0.000003,0.000026,-0.000070,0.000358,0.000355,-0.000045,-0.000222,0.000080,0.000411,-0.000053,0.000246,-0.000020,-0.000347,-0.000091,-0.000163,-0.000088,-0.000393,0.000061,-0.000080,-0.000174,0.000293,0.000195,0.000520,-0.000209,-0.000136,0.000132,-0.000011,0.000229,0.000118,0.000048,-0.000364,-0.000086,0.000132,-0.000239,-0.000057,0.000335,...,0.000065,0.000300,0.000093,0.000132,-0.000295,-0.000085,-0.000021,-0.000104,-0.000416,-0.000232,0.000770,-0.000001,0.000179,0.000478,-0.000638,0.000316,0.000074,-0.000141,-0.000160,-0.000244,0.000245,0.000049,-0.000066,-0.000200,0.000287,-0.000334,0.000011,0.000267,0.000400,0.000119,-0.000107,-0.000318,0.000562,-0.000143,0.000059,0.000312,-0.000340,-0.000120,-0.000072,-0.000053
2,0,"[0.00058013183, 0.00024639844, -0.00029291608,...",0.000580,0.000246,-0.000293,-0.000368,-0.000061,0.000077,-0.000128,0.000394,0.000324,0.000146,-0.000764,0.000209,0.000430,-0.000355,0.000365,0.000145,0.000420,0.000022,0.000399,-0.000136,-0.000035,0.000184,-0.000066,-0.000208,-0.000134,0.000397,0.000325,-0.000472,-0.000361,-0.000285,-0.000079,0.000039,0.000067,-0.000094,-0.000468,-0.000125,0.000156,-0.000210,...,0.000856,0.000428,-0.000600,-0.000294,-0.000177,-0.000084,-0.000168,0.000015,-0.000149,-0.000532,-0.000433,0.000166,-0.000340,0.000197,-0.000167,0.000197,0.000947,-0.000448,0.000002,-0.000165,-0.000015,0.000147,-0.000504,0.000511,-0.000113,-0.000155,0.000022,0.000966,-0.000080,-0.000068,-0.000354,-0.000079,0.000167,-0.000355,-0.000204,-0.000136,-0.000159,-0.000284,0.000594,-0.000063
3,0,"[2.6209871e-05, -0.00026430548, 5.9815775e-06,...",0.000026,-0.000264,0.000006,-0.000156,0.000978,-0.000096,-0.000134,0.000251,0.000501,0.000518,0.000138,0.000336,0.000225,-0.000146,-0.000099,0.000124,0.000292,0.000124,0.000081,-0.000476,0.000140,0.000484,0.000180,-0.000218,0.000198,-0.000316,-0.000153,0.000045,-0.000036,0.000217,-0.000259,-0.000210,-0.000633,-0.000328,-0.000239,-0.000360,0.000270,-0.000556,...,0.000539,-0.000246,-0.000328,0.000375,-0.000641,0.000588,0.000393,-0.000401,0.000433,0.000171,0.000141,0.000070,0.000488,0.000223,0.000067,0.000057,-0.000092,0.000411,0.000285,-0.000189,0.000318,0.000474,0.000163,0.000149,-0.000420,-0.000124,0.000010,-0.000091,0.000567,-0.000551,-0.000282,0.000094,0.000162,0.000151,-0.000093,0.000461,-0.000484,0.000101,0.000171,0.000155
4,0,"[0.00012178716, 0.00052041665, -0.00043820584,...",0.000122,0.000520,-0.000438,-0.000388,-0.000078,-0.000350,-0.000355,0.000373,0.000558,0.000922,-0.000439,-0.000044,-0.000332,0.000194,-0.000535,0.000211,0.000010,-0.000205,-0.000034,0.000306,-0.000012,0.000264,-0.000057,0.000120,0.000162,0.000225,0.000961,-0.000237,0.000094,0.000185,-0.000338,0.000033,0.000125,-0.000084,-0.001020,0.000451,-0.000055,-0.000322,...,-0.000075,0.000917,0.000396,0.000421,-0.000073,0.000635,-0.000008,-0.000320,0.000260,

Will be attempting xgboost here.

In [78]:
from numpy import loadtxt
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [79]:
# split data into X and y
X = df3.iloc[:, 2:]
Y = df3['labelML']

In [80]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.33, random_state=123)

In [81]:
X_train

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,260,261,262,263,264,265,266,267,268,269,270,271,272,273,274,275,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,294,295,296,297,298,299
12429,-0.000030,-0.000018,-0.000231,0.000092,-0.000112,-0.000069,0.000149,-0.000607,0.000013,-0.000085,-4.407407e-05,0.000093,0.000365,-0.000026,0.000148,0.000191,0.000070,-0.000088,0.000417,-0.000034,0.000134,-3.971391e-04,0.000204,-0.000143,0.000276,-0.000295,-0.000078,-0.000190,-0.000046,-0.000203,-0.000175,-0.000130,0.000481,-0.000034,-0.000107,-0.000091,0.000289,-0.000261,-0.000053,-0.000010,...,-0.000165,0.000068,-0.000030,0.000209,-0.000023,-0.000323,-0.000048,-0.000074,0.000171,-0.000017,-0.000228,-0.000301,0.000343,0.000058,-0.000548,-0.000100,0.000150,-0.000337,0.000197,0.000507,0.000165,-0.000092,-0.000028,0.000050,0.000179,-0.000243,-0.000035,0.000217,0.000015,-0.000450,-0.000101,2.761938e-04,0.000183,0.000379,-0.000558,-0.000072,0.000074,0.000188,0.000142,-0.000076
8790,0.000123,0.000927,-0.000421,-0.000670,0.000366,0.000583,-0.000226,0.000048,0.000141,-0.000350,3.063060e-04,0.000335,0.000465,0.000479,0.000028,-0.000564,-0.000060,-0.000421,0.000010,0.000365,-0.000079,1.210747e-04,0.000310,0.000062,0.000456,-0.000292,-0.000372,-0.000359,0.000019,-0.000008,0.000270,0.000379,-0.000226,0.000148,-0.000355,0.000065,0.000323,-0.001025,-0.000060,-0.000221,...,0.000417,0.000608,-0.000219,-0.000301,0.000247,0.000343,-0.000595,0.000004,-0.000404,-0.000090,0.000021,0.000826,-0.000050,-0.000002,0.000062,0.000200,-0.000137,-0.000469,-0.000223,-0.000112,-0.000104,-0.000465,-0.000219,0.000585,0.000035,-0.000022,-0.000161,-0.000120,0.000076,0.000214,0.000018,-2.741157e-04,0.000431,0.000158,0.000367,-0.000368,-0.000370,0.000347,-0.000198,0.000008
14900,-0.000028,0.000216,0.000269,0.000085,0.000127,-0.000323,0.000038,0.000728,0.000077,-0.000064,-7.471413e-07,0.000644,0.000153,-0.000302,-0.000234,0.000185,0.000364,0.000540,0.000084,0.000037,-0.000980,-5.538585e-04,-0.000118,0.000092,0.000566,-0.000160,-0.000076,-0.000070,0.000044,0.000189,-0.000584,0.000047,-0.000104,-0.000497,0.000182,-0.000112,-0.000187,-0.000388,-0.000289,-0.000119,...,0.000264,0.000246,-0.000200,-0.000204,-0.000314,0.000299,0.000346,-0.000554,0.000437,-0.000185,0.000100,0.000414,0.000640,-0.000347,-0.000186,0.000494,0.000382,0.000101,0.000120,0.000122,-0.000271,0.000514,-0.000222,-0.000845,-0.000110,-0.000153,-0.000408,-0.000122,0.000088,0.000514,-0.000363,1.220794e-04,-0.000133,0.001038,0.000626,-0.000265,-0.000217,-0.000152,-0.000381,0.000462
5026,-0.000225,-0.000440,0.000083,-0.000209,0.000198,0.000092,0.000191,0.000725,0.000230,-0.000191,-2.159802e-04,-0.000175,0.000442,0.000138,-0.000273,0.000180,0.000611,0.000616,0.000141,-0.000514,0.000030,1.294562e-04,-0.000011,0.000065,0.000082,-0.000205,0.000369,0.000181,-0.000100,0.000146,-0.000016,0.000211,0.000151,0.000154,-0.000509,0.000208,0.000238,-0.000992,-0.000274,-0.000218,...,-0.000395,0.000093,0.000086,0.000464,0.000267,-0.000107,0.000049,0.000108,0.000493,-0.000194,-0.000124,-0.000064,-0.000105,-0.000085,-0.000045,0.000256,0.000976,0.000373,0.000023,0.000101,0.000273,-0.000356,0.000054,0.000241,-0.000048,-0.000387,0.000210,0.000243,-0.000193,-0.000387,-0.000113,1.340974e-04,0.000029,0.000364,-0.000400,-0.000112,-0.000157,0.000465,0.000516,0.000099
11479,-0.000350,0.000327,0.000068,-0.000200,-0.000272,-0.000301,-0.000092,0.000552,0.000079,0.000521,-9.518621e-05,-0.000213,0.000375,0.000200,0.000103,0.000268,0.000115,0.000915,0.000018,0.000140,0.000265,2.821026e-05,-0.000373,-0.000010,-0.000142,0.000389,-0.000496,0.000202,-0.000051,-0.000366,0.000388,-0.000154,-0.000184,0.000031,-0.000439,-0.000168,0.000349,-0.000123,-0.000173,0.000064,...,-0.000374,-0.000032,-0.000096,0.000576,-0.000132,-0.000840,0.000064,0.000333,0.000258,0.000094,-0.000291,0.000140,0.000175,0.000103,-0.000286,0.000484,0.000242,0.000280,0.000097,0.000084,0.000450,0.000270

In [82]:
# Create arrays for the features and the target: X, y
X, y = df3['prediction'], df3['labelML']
#print(X.columns)

# Create the training and test sets
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=123)

# Instantiate the XGBClassifier: xg_cl
xg_cl = xgb.XGBClassifier(objective='binary:logistic', n_estimators=10, seed=123)

# Fit the classifier to the training set
xg_cl.fit(X_train, y_train)

# Predict the labels of the test set: preds
preds = xg_cl.predict(X_test)

# Compute the accuracy: accuracy
accuracy = float(np.sum(preds == y_test)) / y_test.shape[0]
print("accuracy: %f" % (accuracy))

accuracy: 0.765020


In [83]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_true, y_pred)

#tn, fp, fn, tp = confusion_matrix([0, 1, 0, 1], [1, 1, 1, 0]).ravel()
#>>> (tn, fp, fn, tp)

NameError: ignored

In [ ]:
type(X)

In [ ]:
y = dfready[['labelML']]

In [ ]:
type(y)

Keras NN model